In [1]:
#########################################################################
#Data Collection 
#
# Functions for handling subwatershed characteristics data collection
#
# Authors: Shannon McAvoy (smcavoy@dewberry.com)
#
# Editor: Jason Matney (jmatney@dewberry.com)
#
# Copyright: Dewberry Engineers Inc.
#########################################################################

In [2]:
import pandas as pd
import arcpy
import numpy as np
from numpy import mean
from numpy import std
import os
from arcpy import management
from arcgis.gis import GIS
from time import time

In [3]:
# Login for ArcPro, AGOL or API Server 
#gis = GIS("https://www.arcgis.com", username="dpiazza_dewberry", password="*****")

In [4]:
path = "C:\\Users\jmatney\Documents\GitHub\IndianaRisk\data\Indiana_ML"
hucs = "C:\\Users\jmatney\Documents\GitHub\IndianaRisk\data\shp"

In [5]:
arcpy.env.overwriteOutput = True

In [6]:
st = time()

# Set Local Variables
output_workspace = os.path.join(path, "working_dsn")

arcpy.env.workspace = output_workspace

##### change these based on what subwatersheds you want #####

# all_subwatersheds = r"P:\Temp\McAvoy\ML_DataCollection\Indiana_ML\Indiana_HUC12_groups\1.shp"
all_subwatersheds = os.path.join(hucs, "wbdhu12_a_in.shp")

# All centroids of subwatersheds
all_centroids = os.path.join(hucs, "wbdhu12_a_in_centroids.shp")

#set source for streams data
streams = os.path.join(path, "Indiana_50c_streams_edited\Hydrography_HighRes_FlowLine_NHD_USGS.shp")

#####these are the same for all subwatersheds in Indiana#####

#set source for dem
dem_Indiana = os.path.join(path, "IN_EXTENT_MOSAIC\IN_EXTENT_MOSAIC.tif")

#set source for slope dem
dem_slope = os.path.join(path, "Indiana_SlopeRaster\IN_slope2")

#set source for NHFL Data
nfhl_sfha = os.path.join(path, "NFHL_18_20200310.gdb\S_FLD_HAZ_AR")


#set source for water bodies data 
water_bodies = os.path.join(path, "IndianaMAP_WaterBodies\Water_Bodies_Lakes_LocalRes\Hydrography_LocalRes_WaterbodyDiscrete_NHD_IN.shp")

#set source for dams data 
dams = os.path.join(path, "IndianaMAP_Dams\Dams_IDNR\Dams_IDNR_IN.shp")

#set source for bridges data
bridges = os.path.join(path, "IndianaMAP_Bridges\Bridges_County_INDOT\Bridges_County_INDOT_IN.shp")

#set source for streets data
streets = os.path.join(path, "IndianaMAP_Streets\Streets_Centerlines_IGIO\County_Street_Centerlines_IGIO_IN.gdb\County_Street_Centerlines_IGIO_IN_Dec2019")


#set source for railraods data
railroads = os.path.join(path, "IndianaMAP_Railroads\Railroads_Active_Abandoned_INDOT\Rail_System_Active_Abandoned_INDOT_IN.shp")

#set source for ACS population data
population = os.path.join(path, "Indiana_PopulationData\Indiana_PopulationData.shp")

#set source for ACS median income data 
median_income = os.path.join(path, "Indiana_IncomeData\Indiana_IncomeData.shp")

#set source for county boundary data
county_boundary = os.path.join(path, "MarionCounty_Boundary\Marion_County_Boundary.shp")

#source for Bing building footprints for Indiana (attrubuted with open street maps data)
building_footprints = os.path.join(path, "BuildingFootprints_Indiana\Building_Footprints_Attributed_IN.shp")

#folder with partial duration files, set as workspace temporarily, then reset when done
directory_rainfall = os.path.join(path, "MarionCounty_Rainfall\All_Rainfall_Clipped_IN")

#set source for nlcd land use data
lu_usa = os.path.join(path, r"NLCD_Impervious\NLCD_indiana_polygon.shp")

#set source for impervious indicator data
impervious_usa = os.path.join(path, "NLCD_Impervious\\NLCD_2016_Impervious_L48_20190405_PERCENT\\NLCD_2016_Impervious_L48_20190405.img")



subwatershed_list = []
area_list = []
perimeter_list = []
watershed_length_list = []
elongation_ratio_list = []
shape_factor_list = []
circulatory_ratio_list = []
relief_list = []
relief_ratio_list = []
avg_slope_list = []
drainage_density_list = []
ruggedness_list = []
aae_list = []
buildings_aae_list = []
x_list = []
buildings_x_list = []
water_bodies_list = []
dams_list = []
bridges_list = []
streets_list = []
railroads_list = []
population_list = []
dependent_population_list = []
population_density_list = []
avg_median_income_list = []
housing_density_list = []
population_change_list = []
dist_to_stream_avg_list = []
dist_to_stream_stdev_list = []

lu_21_list = []
lu_22_list = []
lu_23_list = []
lu_24_list = []
lu_41_list = []
lu_82_list = []
impervious_percent_list = []

orb100yr06h_list = []
orb100yr12h_list = []
orb100yr24h_list = []
orb25yr06h_list = []
orb25yr12h_list = []
orb25yr24h_list = []
orb2yr06h_list = []
orb2yr12h_list = []
orb2yr24h_list = []
orb50yr06h_list = []
orb50yr12h_list = []
orb50yr24h_list = []
orb100yr06ha_am_list = []
orb100yr12ha_am_list = []
orb100yr24ha_am_list = []
orb25yr06ha_am_list = []
orb25yr12ha_am_list = []
orb25yr24ha_am_list = []
orb2yr06ha_am_list = []
orb2yr12ha_am_list = []
orb2yr24ha_am_list = []
orb50yr06ha_am_list = []
orb50yr12ha_am_list = []
orb50yr24ha_am_list = []


#########################################################################################################
print(round(((time()-st)/60), 2) , 'minutes to process.')

0.01 minutes to process.


In [7]:
# GLOBAL VARIABLES
LOG_NAME = 'DataCollection_export.log'

In [8]:
st = time()

#search cursor through each row of county subwatersheds file
rows = arcpy.SearchCursor(all_subwatersheds)

for row in rows:
    subwatershed_number = row.getValue("HUC12")
    
    subwatershed_list.append(subwatershed_number)
    
    print("--------------------", subwatershed_number, " is the subwatershed to be worked on ------------------------")
            
    
    #select subwatershed from shapefile with all of them
    subwatershed_selection = arcpy.SelectLayerByAttribute_management(all_subwatersheds, "NEW_SELECTION",
                                                                     "HUC12 = " + "'"+subwatershed_number+"'")
    
    #select centroid from shapefile with all of them
    centroid_selection = arcpy.SelectLayerByAttribute_management(all_centroids, "NEW_SELECTION",
                                                                     "HUC12 = " + "'"+subwatershed_number+"'")
    #copy selected subwatershed to it's own file
    subwatershed = arcpy.CopyFeatures_management(subwatershed_selection, "subwatershed")
    
    #copy selected centroid to it's own file
    centroid = arcpy.CopyFeatures_management(centroid_selection, "centroid")
    
    #clip dem to subwatershed area
    # dem_clip = arcpy.Clip_management(dem_Indiana, "#", "dem_clip.tif", subwatershed, "#" , "ClippingGeometry", "NO_MAINTAIN_EXTENT")
    dem_clip =  os.path.join(path, "working_dsn\dem_clip.tif")
    arcpy.Clip_management(dem_Indiana, "#", dem_clip, subwatershed_selection)
    print("clipped dem to subwatershed area")
    
    #calculate area of subwatershed
    arcpy.AddField_management(subwatershed, "AREA", "DOUBLE")

    area = arcpy.CalculateGeometryAttributes_management(subwatershed, "AREA AREA_GEODESIC", '', 
                                                        "SQUARE_KILOMETERS",
                                                        None)
    
    #print area
    rows = arcpy.SearchCursor(area)

    for row in rows:
        area = row.getValue("AREA")
    
    area_list.append(area)
    print(area, " square kilometers")
    
    
    #calculate perimeter of subwatershed
    arcpy.AddField_management(subwatershed, "PERIMETER", "DOUBLE")
    perimeter = arcpy.CalculateGeometryAttributes_management(subwatershed, "PERIMETER PERIMETER_LENGTH_GEODESIC",
                                                             "KILOMETERS")
    
    print("perimeter calculated")

    #print perimeter
    rows = arcpy.SearchCursor(perimeter)

    for row in rows:
        perimeter = row.getValue("PERIMETER")
   
    perimeter_list.append(perimeter)
    print(perimeter, " kilometers")
    


    #get average slope using zonal statistics from slope dem
    avg_slope = arcpy.sa.ZonalStatistics(subwatershed, "FID", dem_slope, "MEAN")

    #get slope value
    avg_slope_result = arcpy.GetRasterProperties_management(avg_slope, "MAXIMUM")

    avg_slope_value = avg_slope_result.getOutput(0)
    
    avg_slope_list.append(avg_slope_value)
    
    print(avg_slope_value, " = avg slope (%)")

    
    #calculate circulatory ratio
    #ratio of area to the area of a circle having equal perimeter as the perimeter of drainage basin

    #area of a circle with same perimeter as above
    #C = 2(pi)r
    #r = C/(2pi)
    #A = (pi)r^2
    circle_radius = perimeter/(2*numpy.pi)
    print(circle_radius, "is the radius of a circle with the same perimeter.")
    circle_area = (numpy.pi*(circle_radius**2))
    print(circle_area, "is the area of a circle with the same perimeter.")

    #ratio of subwatershed perimeter to circle circumference
    circulatory_ratio = area / circle_area
    
    circulatory_ratio_list.append(circulatory_ratio)
    print(circulatory_ratio, " is the circulatory ratio")
    
    
    #calculate relief
    #elevation difference before basin outlet and highest point located in the perimeter of basin

    #find highest point on perimeter

    #use Raster Domain tool to get z-enabled polyline of perimeter
    perimeter_polyline = arcpy.RasterDomain_3d(dem_clip, "perimeter_polyline.shp", "LINE")
    print("created 3d polyline of subwatershed perimeter")

    #Convert each vertices of the polyline into points
    perimeter_points = arcpy.FeatureVerticesToPoints_management(perimeter_polyline, "perimeter_points.shp")
    print("created perimeter points")

    #get Z values into the attribute table for the points
    perimeter_points_Z = arcpy.AddZInformation_3d(perimeter_points, "Z")
    print("added z information")

    #get the max value from all of the points
    perimeter_stats = arcpy.Statistics_analysis(perimeter_points_Z, "perimeter_stats", [["Z", "MAX"], ["Z", "MIN"]])

    # Get a list of field names to display
    field_names = [i.name for i in arcpy.ListFields(perimeter_stats) if i.type != 'OID']

    # Open a cursor to extract results from stats table
    cursor = arcpy.da.SearchCursor(perimeter_stats, field_names)

    # Create a pandas dataframe to display results
    df = pd.DataFrame(data=[row for row in cursor],
                          columns=field_names)

    print(df)

    #get the values for max Z and min Z into a format to use them

    rows = arcpy.SearchCursor(perimeter_stats)

    for row in rows:
        max_z = row.getValue("MAX_Z")
        min_z = row.getValue("MIN_Z")


    print(max_z)
    print(min_z)

    relief = max_z - min_z
    print("The relief of the subwatershed is: ", relief," meters.")
    
    relief_list.append(relief)
    
    
    
    #find area covered by A and AE zones and X NFHL Zones
    
    #clip to subwatershed area
    nfhl_sfha_clip = arcpy.Clip_analysis(nfhl_sfha, subwatershed, "nfhl_sfha.shp")

    #select Zone A and Zone AE
    nfhl_sfha_aae_selection = arcpy.management.SelectLayerByAttribute(nfhl_sfha_clip, "NEW_SELECTION", 
                                                       "FLD_ZONE = 'AE' Or FLD_ZONE = 'A'", None)

    #copy features to new feature class
    nfhl_sfha_aae = arcpy.CopyFeatures_management(nfhl_sfha_aae_selection, "nfhl_sfha_aae")

    #add new field for area
    nfhl_sfha_aae_addfield = arcpy.AddField_management(nfhl_sfha_aae, "AREA", "DOUBLE")

    #calculate the area for each of the features
    aae_area = arcpy.CalculateGeometryAttributes_management(nfhl_sfha_aae_addfield, "AREA AREA_GEODESIC", '', 
                                                        "SQUARE_KILOMETERS",
                                                        None)
    #get the sum of all of the areas
    aae_area_list = []  

    rows = arcpy.SearchCursor(aae_area)  
    for row in rows:  
        aae_area_feature = row.getValue("AREA")  
        aae_area_list.append(aae_area_feature)  

    aae_area_sum = sum(aae_area_list)
    
    aae_list.append(aae_area_sum)

    print(aae_area_sum, " = area of all AE and A zones in the subwatershed (square km)")



    #select Zone X, 0.2% chance flood area
    nfhl_sfha_x_selection = arcpy.management.SelectLayerByAttribute(nfhl_sfha_clip, "NEW_SELECTION", 
                                                       "FLD_ZONE = 'X' And ZONE_SUBTY = '0.2 PCT ANNUAL CHANCE FLOOD HAZARD'",
                                                                      None)

    #copy features to new feature class
    nfhl_sfha_x = arcpy.CopyFeatures_management(nfhl_sfha_x_selection, "nfhl_sfha_x")

    #add new field for area
    nfhl_sfha_x_addfield = arcpy.AddField_management(nfhl_sfha_x, "AREA", "DOUBLE")

    #calculate the area for each of the features
    x_area = arcpy.CalculateGeometryAttributes_management(nfhl_sfha_x_addfield, "AREA AREA_GEODESIC", '', 
                                                        "SQUARE_KILOMETERS",
                                                        None)
    #get the sum of all of the areas
    x_area_list = []  

    rows = arcpy.SearchCursor(x_area)  
    for row in rows:  
        x_area_feature = row.getValue("AREA")  
        x_area_list.append(x_area_feature)  

    x_area_sum = sum(x_area_list)
    
    x_list.append(x_area_sum)

    print(x_area_sum, " = area of all X zones, 0.2PCT zone subtype in the subwatershed (square km)")


    #calculate area covered by lakes/reserviors 
    
    #clip to subwatershed area
    water_bodies_clip = arcpy.Clip_analysis(water_bodies, subwatershed, "water_bodies.shp")

    #copy features to new feature class
    water_bodies_copy = arcpy.CopyFeatures_management(water_bodies_clip, "water_bodies_copy.shp")

    #add new field for area
    water_bodies_addfield = arcpy.AddField_management(water_bodies_copy, "AREA", "DOUBLE")

    #calculate the area for each of the features
    water_bodies_area = arcpy.CalculateGeometryAttributes_management(water_bodies_addfield, "AREA AREA_GEODESIC", '', 
                                                        "SQUARE_KILOMETERS",
                                                        None)
    #get the sum of all of the areas
    water_bodies_area_list = []  

    rows = arcpy.SearchCursor(water_bodies_area)  
    for row in rows:  
        wb_area_values = row.getValue("AREA")  
        water_bodies_area_list.append(wb_area_values)  

    water_bodies_area_sum = sum(water_bodies_area_list)
    
    water_bodies_list.append(water_bodies_area_sum)
    print(water_bodies_area_sum, " = area of all water bodies in the subwatershed (square km)")
    
    
    #count all of the dams in the subwatershed

    #clip dams to subwatershed
    dams_clip = arcpy.Clip_analysis(dams, subwatershed, "dams.shp")

    #count number of dam points
    dams_count = arcpy.GetCount_management(dams_clip)
    
    dams_list.append(dams_count)

    print(dams_count, " = number of dams in the subwatershed")

    
    
    #count all of the bridges in the subwatershed

    #clip bridges to subwatershed
    bridges_clip = arcpy.Clip_analysis(bridges, subwatershed, "bridges.shp")

    #count number of dam points
    bridges_count = arcpy.GetCount_management(bridges_clip)
    
    bridges_list.append(bridges_count)

    print(bridges_count, " = number of bridges in the subwatershed")
    
    
    
    #calculate the kilometers of streets in the subwatershed

    #clip streets to subwatershed area 
    streets_clip = arcpy.Clip_analysis(streets, subwatershed, "streets.shp")

    #add field to calculate length of each street
    streets_addfield = arcpy.AddField_management(streets_clip, "LENGTH_KM", "DOUBLE")

    #calculate the area for each of the features
    streets_length = arcpy.CalculateGeometryAttributes_management(streets_addfield, "LENGTH_KM LENGTH_GEODESIC", 
                                                                  'KILOMETERS')
    #get the sum of all of the areas
    streets_lengths_list = []  

    rows = arcpy.SearchCursor(streets_length)  
    for row in rows:  
        streets_lengths_values = row.getValue("LENGTH_KM")  
        streets_lengths_list.append(streets_lengths_values)  

    streets_length_sum = sum(streets_lengths_list)
    
    streets_list.append(streets_length_sum)

    print(streets_length_sum, " = sum of all streets in the subwatershed (km)")

    
    #calculate km of railroads in subwatershed

    #clip railroads to subwatershed area 
    railroads_clip = arcpy.Clip_analysis(railroads, subwatershed, "railroads.shp")

    #add field to calculate length of each railroad
    railroads_addfield = arcpy.AddField_management(railroads_clip, "LENGTH_KM", "DOUBLE")

    #calculate the area for each of the features
    railroads_length = arcpy.CalculateGeometryAttributes_management(railroads_addfield, "LENGTH_KM LENGTH_GEODESIC", 'KILOMETERS', 
                                                        "",
                                                        None)
    #get the sum of all of the areas
    railroads_lengths_list = []  

    rows = arcpy.SearchCursor(railroads_length)  
    for row in rows:  
        railroads_lengths_values = row.getValue("LENGTH_KM")  
        railroads_lengths_list.append(railroads_lengths_values)  

    railroads_length_sum = sum(railroads_lengths_list)
    
    railroads_list.append(railroads_length_sum)

    print(railroads_length_sum, " = sum of all railroads in the subwatershed (km)")   
    
    
    #ACS population data - 5 year estimates (2014-2018) gotten from ESRI Living Atlas Data
    #data is chosen to be on census tract level

   
    #clip to subwatershed
    population_clip = arcpy.Clip_analysis(population, subwatershed, "population.shp")

    #get total population
    #field = B01001_001E = Total Population (alias)
    #when clipped, field name changes to B01001_001

    #get the sum of all of the populations in each tract
    total_pop_list = []  

    rows = arcpy.SearchCursor(population_clip)  
    for row in rows:  
        total_pop_value = row.getValue('B01001_001') 
        total_pop_list.append(total_pop_value)  

    total_pop_sum = sum(total_pop_list)
    
    population_list.append(total_pop_sum)
    print(total_pop_sum, " = total population in the subwatershed") 
    
    #get the average of percentages of dependent age groups in each tract
    #field = B01001_calc_pctDependE = Percent of Population in Dependent Age Groups (under 18 and 65+) (alias)
    #when clipped field changes to B01001_61
    dependent_pop_list = []  

    rows = arcpy.SearchCursor(population_clip)  
    for row in rows:  
        dependent_pop_value = row.getValue('B01001__61') 
        dependent_pop_list.append(dependent_pop_value)  

    dependent_pop_avg_pct = mean(dependent_pop_list)
    
    dependent_population_list.append(dependent_pop_avg_pct)

    print(dependent_pop_avg_pct, " = total percent of dependent population in the subwatershed") 
    
    #find population density
    population_density = total_pop_sum / area
    
    population_density_list.append(population_density)
    
    print(population_density, " = population density of subwatershed (people/square km)")

    
    #ACS population data - 5 year estimates (2014-2018) gotten from ESRI Living Atlas Data
    #data is chosen to be on census tract level

    #clip to subwatershed
    median_income_clip = arcpy.Clip_analysis(median_income, subwatershed, "median_income.shp")

    #get average median income 
    #field = B19049_001 = Median Household Income in past 12 months 
    #^(inflation-adjusted dollars to last year of 5-year range) (alias)

    #get the sum of all of the populations in each tract
    total_median_income_list = []  

    rows = arcpy.SearchCursor(median_income_clip)  
    for row in rows:  
        median_income_value = row.getValue('B19049_001') 
        total_median_income_list.append(median_income_value)  

    median_income_average = mean(total_median_income_list)
    
    avg_median_income_list.append(median_income_average)

    print(median_income_average, " = average median income in the subwatershed") 

    
    #get housing density

    #clip building footprints to subwatershed area
    building_footprints_clip = arcpy.Clip_analysis(building_footprints, subwatershed, "building_footprints.shp")

    #select buildings that are marked residential
    building_footprints_residential = arcpy.SelectLayerByAttribute_management(building_footprints_clip, "NEW_SELECTION",
                                                                         "RES_NONRES = 'Res'")

    #get count of how many buildings there are 
    buildings_count = arcpy.GetCount_management(building_footprints_residential)

    buildings_count_number = buildings_count.getOutput(0)
    print(buildings_count_number, " = number of residential building footprints in the subwatershed")

    #divide number of buildings by subwatershed area
    housing_density = int(buildings_count_number) / area
    
    housing_density_list.append(housing_density)

    print(housing_density, " = housing density (buildings per square km)")
    
    
   #find total population from the 2013 5-year ACS estimates
    #get the sum of all of the 2013 populations in each tract
    #field = DP05_0001E = TotalPopulation from the 2013 ACS 5yr estimates
    total_pop_list_2013 = []  

    rows = arcpy.SearchCursor(population_clip)  
    for row in rows:  
        total_pop_value_2013 = row.getValue('DP05_0001E')
        if total_pop_value_2013 != '0':
            total_pop_list_2013.append(int(total_pop_value_2013))  

    total_pop_sum_2013 = sum( total_pop_list_2013)

    print(total_pop_sum_2013, " = total 2013 population in the subwatershed") 

    #find population change between 2018 and 2013
    population_change = total_pop_sum - total_pop_sum_2013
    
    population_change_list.append(population_change)
    print(population_change, " = population change between 2013 and 2018")

    
    
   
    #calculate drainage density
    #the total length of all streams and tributaries divided by basin area

    #all streams in subwatershed
    streams_clip = arcpy.Clip_analysis(streams, subwatershed, "streams")
    print("streams clipped")

    #add field for length
    streams_addfield = arcpy.AddField_management(streams_clip, "LENGTH", "DOUBLE")

    #calculate geometry 
    streams_calculate = arcpy.CalculateGeometryAttributes_management(streams_addfield, [["LENGTH", "LENGTH_GEODESIC"]], 
                                                                     "KILOMETERS")
    print("length calculated")

    #get the sum of all of the areas
    stream_length_list = []  

    rows = arcpy.SearchCursor(streams_calculate)  
    for row in rows:  
        stream_length = row.getValue("LENGTH")  
        stream_length_list.append(stream_length)  

    stream_length_sum = sum(stream_length_list)

    print(stream_length_sum, " = length of all streams in subwatershed")

    print(area, " = subwatershed area")

    drainage_density = stream_length_sum / area
    
    drainage_density_list.append(drainage_density)

    print(drainage_density, "is the drainage density (streams/km)")

    #find watershed length
    #watershed length = distance from outlet to watershed boundary along the main channel
    #we are assuming that the longest stream above is the main channel

    watershed_length = max(stream_length_list)
    
    watershed_length_list.append(watershed_length)

    print(watershed_length, " = watershed length (longest stream in subwatershed)")
    
    
    #calculate shape factor 
    #watershed length squared divided by watershed area

    shape_factor = (watershed_length**2) / area
    
    shape_factor_list.append(shape_factor)

    print(shape_factor, " is the shape factor.")
    
    
    
    #calculate relief ratio
    #relief divided by watershed length
    #length is is kilometers, convert to meters

    watershed_length_meters = watershed_length * 1000

    relief_ratio = relief / watershed_length_meters
    
    relief_ratio_list.append(relief_ratio)

    print(relief_ratio, " = relief ratio")
    
    
    #calculate ruggedness number
    #product of relief and drainage density

    #relief is in meters, convert first to km
    relief_km = relief / 1000
    print(relief_km, " is the relief in km")

    ruggedness = relief_km * drainage_density
    
    ruggedness_list.append(ruggedness)
    print(ruggedness, " is the ruggedness number")
    
    
    #calculate elongation ratio
    #ratio of diameter of a circle having the same area as the basin to the max basin length
    

    #diameter of circle with same area
    #A = (pi)r^2
    #r = sqrt(A/pi)
    #d = r*2

    radius = numpy.sqrt(area/numpy.pi)
    print(radius, " miles is the radius of a circle with the same area.")

    diameter = radius*2
    print(diameter, " miles is the diameter of a circle with the same area.")

    elongation_ratio = diameter / watershed_length
    
    elongation_ratio_list.append(elongation_ratio)
    
    
    #get number of buildings inside the aae zone
    
    buildings_aae_select = arcpy.SelectLayerByLocation_management(building_footprints_clip, "INTERSECT", nfhl_sfha_aae )
    
    buildings_aae = arcpy.CopyFeatures_management(buildings_aae_select, "buildings_aae.shp")
    
    buildings_aae_count = arcpy.GetCount_management(buildings_aae)
    
    buildings_aae_list.append(buildings_aae_count)
    
    #get number of buildings inside the x zone
    
    buildings_x_select = arcpy.SelectLayerByLocation_management(building_footprints_clip, "INTERSECT", nfhl_sfha_x )
    
    buildings_x = arcpy.CopyFeatures_management(buildings_x_select, "buildings_x.shp")
    
    buildings_x_count = arcpy.GetCount_management(buildings_x)
    
    buildings_x_list.append(buildings_x_count)

    #get area of various land use codes
    #clip usa land use polygon to the subwatershed
    lu_subwatershed = arcpy.Clip_analysis(lu_usa, subwatershed, "lu_subwatershed.shp")
    
    lu_subwatershed_area = arcpy.AddGeometryAttributes_management(lu_subwatershed, "AREA_GEODESIC", '',
                                                      "SQUARE_KILOMETERS")
    
    
    #lu 21 = developed open space
    lu_21_select = arcpy.SelectLayerByAttribute_management(lu_subwatershed_area, "NEW_SELECTION",
                                                                         "gridcode = 21")

    lu_21 = arcpy.CopyFeatures_management(lu_21_select, "lu_21.shp")
    
    #get the sum of all of the areas
    lu_21_polygons_list = []  

    rows = arcpy.SearchCursor(lu_21)  
    for row in rows:  
        lu_21_polygon = row.getValue("AREA_GEO")  
        lu_21_polygons_list.append(lu_21_polygon)  

    lu_21_area_sum = sum(lu_21_polygons_list)
    
    lu_21_list.append(lu_21_area_sum)
    
    
    
    #lu 22 = developed low intensity
    lu_22_select = arcpy.SelectLayerByAttribute_management(lu_subwatershed_area, "NEW_SELECTION",
                                                                         "gridcode = 22")

    lu_22 = arcpy.CopyFeatures_management(lu_22_select, "lu_22.shp")
    
    #get the sum of all of the areas
    lu_22_polygons_list = []  

    rows = arcpy.SearchCursor(lu_22)  
    for row in rows:  
        lu_22_polygon = row.getValue("AREA_GEO")  
        lu_22_polygons_list.append(lu_22_polygon)  

    lu_22_area_sum = sum(lu_22_polygons_list)
    
    lu_22_list.append(lu_22_area_sum)
    
    
    #lu 23 = developed medium intensity
    lu_23_select = arcpy.SelectLayerByAttribute_management(lu_subwatershed_area, "NEW_SELECTION",
                                                                         "gridcode = 23")

    lu_23 = arcpy.CopyFeatures_management(lu_23_select, "lu_23.shp")
    
    #get the sum of all of the areas
    lu_23_polygons_list = []  

    rows = arcpy.SearchCursor(lu_23)  
    for row in rows:  
        lu_23_polygon = row.getValue("AREA_GEO")  
        lu_23_polygons_list.append(lu_23_polygon)  

    lu_23_area_sum = sum(lu_23_polygons_list)
    
    lu_23_list.append(lu_23_area_sum)
    
    
    
    #lu 24 = developed high intensity
    lu_24_select = arcpy.SelectLayerByAttribute_management(lu_subwatershed_area, "NEW_SELECTION",
                                                                         "gridcode = 24")

    lu_24 = arcpy.CopyFeatures_management(lu_24_select, "lu_24.shp")
    
    #get the sum of all of the areas
    lu_24_polygons_list = []  

    rows = arcpy.SearchCursor(lu_24)  
    for row in rows:  
        lu_24_polygon = row.getValue("AREA_GEO")  
        lu_24_polygons_list.append(lu_24_polygon)  

    lu_24_area_sum = sum(lu_24_polygons_list)
    
    lu_24_list.append(lu_24_area_sum)
    
    
    
    #lu 41 = deciduous forest
    lu_41_select = arcpy.SelectLayerByAttribute_management(lu_subwatershed_area, "NEW_SELECTION",
                                                                         "gridcode = 41")

    lu_41 = arcpy.CopyFeatures_management(lu_41_select, "lu_41.shp")
    
    #get the sum of all of the areas
    lu_41_polygons_list = []  

    rows = arcpy.SearchCursor(lu_41)  
    for row in rows:  
        lu_41_polygon = row.getValue("AREA_GEO")  
        lu_41_polygons_list.append(lu_41_polygon)  

    lu_41_area_sum = sum(lu_41_polygons_list)
    
    lu_41_list.append(lu_41_area_sum)
    
    
    #lu 82 = cultivated crops
    lu_82_select = arcpy.SelectLayerByAttribute_management(lu_subwatershed_area, "NEW_SELECTION",
                                                                         "gridcode = 82")

    lu_82 = arcpy.CopyFeatures_management(lu_82_select, "lu_82.shp")
    
    #get the sum of all of the areas
    lu_82_polygons_list = []  

    rows = arcpy.SearchCursor(lu_82)  
    for row in rows:  
        lu_82_polygon = row.getValue("AREA_GEO")  
        lu_82_polygons_list.append(lu_82_polygon)  

    lu_82_area_sum = sum(lu_82_polygons_list)
    
    lu_82_list.append(lu_82_area_sum)
    
    
    
    #get percent impervious indicator for subwatershed area
    avg_impervious_pct = arcpy.sa.ZonalStatistics(subwatershed, "FID", impervious_usa, "MEAN" )
    
    #get avg value  value
    avg_impervious_result = arcpy.GetRasterProperties_management(avg_impervious_pct, "MAXIMUM")

    avg_impervious_pct_value = avg_impervious_result.getOutput(0)
    print(avg_impervious_pct_value, " = avg impervious percent ")
    
    impervious_percent_list.append(avg_impervious_pct_value)
    
    
    
    #get distance from residential buildings to streams

    building_footprints_residential_copy = arcpy.CopyFeatures_management(building_footprints_residential, "buildings_res.shp")


    #project buildings and streams so they are in the same GCS
    output_coord_system = arcpy.SpatialReference(r"P:\Temp\McAvoy\ML_DataCollection\Marion_County_Data\NAD1983_ProjectionFile.prj")

    streams_project = arcpy.Project_management(streams_clip, "streams_project.shp",
                                              output_coord_system)

    buildings_project = arcpy.Project_management(building_footprints_residential_copy, "buildings_res_project.shp",
                                              output_coord_system)

    #use near tool to get distance to steams

    buildings_near = arcpy.Near_analysis(buildings_project, streams_project, "", "LOCATION", "", "GEODESIC")

    #get all the distances in to the streams
    dist_to_stream_list = []
    rows = arcpy.SearchCursor(buildings_near)  
    for row in rows:  
        dist_to_stream = row.getValue("NEAR_DIST")  
        dist_to_stream_list.append(dist_to_stream)  

    dist_to_stream_avg = mean(dist_to_stream_list)

    dist_to_stream_stdev = std(dist_to_stream_list)

    print("Distance to stream avg", dist_to_stream_avg)
    print("Distance to stream std", dist_to_stream_stdev)
    
    dist_to_stream_avg_list.append(dist_to_stream_avg)
    dist_to_stream_stdev_list.append(dist_to_stream_stdev)

    
 ## LOOP    
    
    #loop through all rainfall rasters in the same folder 

   
    arcpy.env.workspace = directory_rainfall

    rasters = arcpy.ListRasters("orb*")

    for raster in rasters:

        raster_name = os.path.basename(raster).rstrip(os.path.splitext(raster)[1])
        print("Raster name: ", raster_name)

        #get average rainfall for duration using zonal statistics
        #elevRaster = arcpy.sa.Raster(r"C:\Users\jmatney\Documents\GitHub\IndianaRisk\data\Indiana_ML\working_dsn\dem_clip.tif")

        centroid_rainfall = arcpy.sa.ExtractValuesToPoints(centroid, raster, r"C:\Users\jmatney\Documents\GitHub\IndianaRisk\data\shp\Centroid_Rainfall.shp", "NONE", "VALUE_ONLY")

        # Open a cursor on some fields in a table  
        rainfall_value = arcpy.da.SearchCursor(centroid_rainfall, ("RASTERVALU",)).next()[0]
        

#         avg_rainfall = arcpy.sa.ZonalStatistics(subwatershed, "FID", raster, "MEAN" )

#         #get avg value  value
#         avg_result = arcpy.GetRasterProperties_management(avg_rainfall, "MAXIMUM")

        #rainfall_value = avg_result.getOutput(0)
        print(rainfall_value, " = avg rainfall in inches *1000")

        if raster_name == "orb100yr06h":
            orb100yr06h_list.append(rainfall_value)

        elif raster_name == "orb100yr12h":
            orb100yr12h_list.append(rainfall_value)

        elif raster_name == "orb100yr24h":
            orb100yr24h_list.append(rainfall_value)

        elif raster_name == "orb25yr06h":
            orb25yr06h_list.append(rainfall_value)

        elif raster_name == "orb25yr12h":
            orb25yr12h_list.append(rainfall_value)

        elif raster_name == "orb25yr24h":
            orb25yr24h_list.append(rainfall_value)

        elif raster_name == "orb2yr06h":
            orb2yr06h_list.append(rainfall_value)

        elif raster_name == "orb2yr12h":
            orb2yr12h_list.append(rainfall_value)

        elif raster_name == "orb2yr24h":
            orb2yr24h_list.append(rainfall_value)

        elif raster_name == "orb50yr06h":
            orb50yr06h_list.append(rainfall_value)

        elif raster_name == "orb50yr12h":
            orb50yr12h_list.append(rainfall_value)

        elif raster_name == "orb50yr24h":
            orb50yr24h_list.append(rainfall_value)

        elif raster_name == "orb100yr06ha_am":
            orb100yr06ha_am_list.append(rainfall_value)

        elif raster_name == "orb100yr12ha_am":
            orb100yr12ha_am_list.append(rainfall_value)

        elif raster_name == "orb100yr24ha_am":
            orb100yr24ha_am_list.append(rainfall_value)

        elif raster_name == "orb25yr06ha_am":
            orb25yr06ha_am_list.append(rainfall_value)

        elif raster_name == "orb25yr12ha_am":
            orb25yr12ha_am_list.append(rainfall_value)

        elif raster_name == "orb25yr24ha_am":
            orb25yr24ha_am_list.append(rainfall_value)

        elif raster_name == "orb2yr06ha_am":
            orb2yr06ha_am_list.append(rainfall_value)

        elif raster_name == "orb2yr12ha_am":
            orb2yr12ha_am_list.append(rainfall_value)

        elif raster_name == "orb2yr24ha_am":
            orb2yr24ha_am_list.append(rainfall_value)

        elif raster_name == "orb50yr06ha_am":
            orb50yr06ha_am_list.append(rainfall_value)

        elif raster_name == "orb50yr12ha_am":
            orb50yr12ha_am_list.append(rainfall_value)

        elif raster_name == "orb50yr24ha_am":
            orb50yr24ha_am_list.append(rainfall_value)

        else:
            continue 
    
    
    
    #set workspace environment back to the newly created folder
    arcpy.env.workspace = output_workspace
    print(round(((time()-st)/60), 2) , 'minutes to process.')  
    

    
    
#########################################################################################


    
    
print(round(((time()-st)/60), 2) , 'minutes to process.')    
    

-------------------- 051202081102  is the subwatershed to be worked on ------------------------
clipped dem to subwatershed area
57.0758769198  square kilometers
perimeter calculated
42.3107896964  kilometers
6.8894152641296  = avg slope (%)
6.733971326303694 is the radius of a circle with the same perimeter.
142.4598223044117 is the area of a circle with the same perimeter.
0.40064543108750234  is the circulatory ratio
created 3d polyline of subwatershed perimeter
created perimeter points
added z information
   FID  FREQUENCY  MAX_Z  MIN_Z
0    0       4549  199.0  135.0
199.0
135.0
The relief of the subwatershed is:  64.0  meters.
3.00603548508  = area of all AE and A zones in the subwatershed (square km)
0  = area of all X zones, 0.2PCT zone subtype in the subwatershed (square km)
3.075746451742836  = area of all water bodies in the subwatershed (square km)
1  = number of dams in the subwatershed
4  = number of bridges in the subwatershed
101.17589538155913  = sum of all streets in 

4.0349255403684  = area of all AE and A zones in the subwatershed (square km)
0  = area of all X zones, 0.2PCT zone subtype in the subwatershed (square km)
1.2337074094435083  = area of all water bodies in the subwatershed (square km)
0  = number of dams in the subwatershed
2  = number of bridges in the subwatershed
99.01391278712157  = sum of all streets in the subwatershed (km)
13.5445434422  = sum of all railroads in the subwatershed (km)
7525  = total population in the subwatershed
41.46666666666666  = total percent of dependent population in the subwatershed
104.64308544461605  = population density of subwatershed (people/square km)
56771.333333333336  = average median income in the subwatershed
526  = number of residential building footprints in the subwatershed
7.314586437723328  = housing density (buildings per square km)
7956  = total 2013 population in the subwatershed
-431  = population change between 2013 and 2018
streams clipped
length calculated
85.33081912175096  = lengt

61766.2  = average median income in the subwatershed
1938  = number of residential building footprints in the subwatershed
28.473592121805737  = housing density (buildings per square km)
24885  = total 2013 population in the subwatershed
38  = population change between 2013 and 2018
streams clipped
length calculated
39.14799684349703  = length of all streams in subwatershed
68.0630667079  = subwatershed area
0.5751723913867249 is the drainage density (streams/km)
5.45414611068  = watershed length (longest stream in subwatershed)
0.4370609676509476  is the shape factor.
0.0027502013506069903  = relief ratio
0.015  is the relief in km
0.008627585870800874  is the ruggedness number
4.654583441846478  miles is the radius of a circle with the same area.
9.309166883692956  miles is the diameter of a circle with the same area.
4.3539733886719  = avg impervious percent 
Distance to stream avg 682.3337172921132
Distance to stream std 572.0199395286713
Raster name:  orb100yr06h
4746  = avg rainf

8.4859561920166  = avg impervious percent 
Distance to stream avg 495.91729537074366
Distance to stream std 382.2424920541338
Raster name:  orb100yr06h
4361  = avg rainfall in inches *1000
Raster name:  orb100yr06ha_am
4343  = avg rainfall in inches *1000
Raster name:  orb100yr12h
5014  = avg rainfall in inches *1000
Raster name:  orb100yr12ha_am
4993  = avg rainfall in inches *1000
Raster name:  orb100yr24h
5648  = avg rainfall in inches *1000
Raster name:  orb100yr24ha_am
5625  = avg rainfall in inches *1000
Raster name:  orb25yr06h
3366  = avg rainfall in inches *1000
Raster name:  orb25yr06ha_am
3352  = avg rainfall in inches *1000
Raster name:  orb25yr12h
3871  = avg rainfall in inches *1000
Raster name:  orb25yr12ha_am
3855  = avg rainfall in inches *1000
Raster name:  orb25yr24h
4436  = avg rainfall in inches *1000
Raster name:  orb25yr24ha_am
4418  = avg rainfall in inches *1000
Raster name:  orb2yr06h
1891  = avg rainfall in inches *1000
Raster name:  orb2yr06ha_am
1741  = avg

3847  = avg rainfall in inches *1000
Raster name:  orb25yr12h
4514  = avg rainfall in inches *1000
Raster name:  orb25yr12ha_am
4495  = avg rainfall in inches *1000
Raster name:  orb25yr24h
5401  = avg rainfall in inches *1000
Raster name:  orb25yr24ha_am
5379  = avg rainfall in inches *1000
Raster name:  orb2yr06h
2250  = avg rainfall in inches *1000
Raster name:  orb2yr06ha_am
2071  = avg rainfall in inches *1000
Raster name:  orb2yr12h
2659  = avg rainfall in inches *1000
Raster name:  orb2yr12ha_am
2448  = avg rainfall in inches *1000
Raster name:  orb2yr24h
3158  = avg rainfall in inches *1000
Raster name:  orb2yr24ha_am
2907  = avg rainfall in inches *1000
Raster name:  orb50yr06h
4385  = avg rainfall in inches *1000
Raster name:  orb50yr06ha_am
4367  = avg rainfall in inches *1000
Raster name:  orb50yr12h
5102  = avg rainfall in inches *1000
Raster name:  orb50yr12ha_am
5081  = avg rainfall in inches *1000
Raster name:  orb50yr24h
6123  = avg rainfall in inches *1000
Raster name

...

perimeter calculated
47.8257229499  kilometers
2.7901136875153  = avg slope (%)
7.611700214420087 is the radius of a circle with the same perimeter.
182.01753281627475 is the area of a circle with the same perimeter.
0.4178116873255422  is the circulatory ratio
created 3d polyline of subwatershed perimeter
created perimeter points
added z information
   FID  FREQUENCY  MAX_Z  MIN_Z
0    0       5283  255.0  255.0
255.0
255.0
The relief of the subwatershed is:  0.0  meters.
2.607652349022  = area of all AE and A zones in the subwatershed (square km)
0  = area of all X zones, 0.2PCT zone subtype in the subwatershed (square km)
6.088149266875582  = area of all water bodies in the subwatershed (square km)
0  = number of dams in the subwatershed
0  = number of bridges in the subwatershed
67.46996005412824  = sum of all streets in the subwatershed (km)
0  = sum of all railroads in the subwatershed (km)
6384  = total population in the subwatershed
43.2  = total percent of dependent population

1.0871024982982196  = area of all water bodies in the subwatershed (square km)
0  = number of dams in the subwatershed
41  = number of bridges in the subwatershed
323.50944458018046  = sum of all streets in the subwatershed (km)
16.930286834621  = sum of all railroads in the subwatershed (km)
86562  = total population in the subwatershed
40.239999999999995  = total percent of dependent population in the subwatershed
884.6069230699903  = population density of subwatershed (people/square km)
79536.0  = average median income in the subwatershed
6506  = number of residential building footprints in the subwatershed
66.48705715548806  = housing density (buildings per square km)
78397  = total 2013 population in the subwatershed
8165  = population change between 2013 and 2018
streams clipped
length calculated
93.55245715719536  = length of all streams in subwatershed
97.8536316442  = subwatershed area
0.9560448149472481 is the drainage density (streams/km)
6.62616030911  = watershed length (l

518  = number of residential building footprints in the subwatershed
6.10671534831479  = housing density (buildings per square km)
20479  = total 2013 population in the subwatershed
-293  = population change between 2013 and 2018
streams clipped
length calculated
73.2824562201832  = length of all streams in subwatershed
84.8246512985  = subwatershed area
0.8639287647915046 is the drainage density (streams/km)
3.43874782237  = watershed length (longest stream in subwatershed)
0.1394050715780959  is the shape factor.
0.009014909380192546  = relief ratio
0.031  is the relief in km
0.026781791708536642  is the ruggedness number
5.196202950271021  miles is the radius of a circle with the same area.
10.392405900542043  miles is the diameter of a circle with the same area.
0.51455217599869  = avg impervious percent 
Distance to stream avg 471.3519658633469
Distance to stream std 403.99589288645
Raster name:  orb100yr06h
4644  = avg rainfall in inches *1000
Raster name:  orb100yr06ha_am
4625  

0.71577817201614  = avg impervious percent 
Distance to stream avg 366.01484821694993
Distance to stream std 229.48950452257893
Raster name:  orb100yr06h
4257  = avg rainfall in inches *1000
Raster name:  orb100yr06ha_am
4239  = avg rainfall in inches *1000
Raster name:  orb100yr12h
4860  = avg rainfall in inches *1000
Raster name:  orb100yr12ha_am
4840  = avg rainfall in inches *1000
Raster name:  orb100yr24h
5517  = avg rainfall in inches *1000
Raster name:  orb100yr24ha_am
5494  = avg rainfall in inches *1000
Raster name:  orb25yr06h
3346  = avg rainfall in inches *1000
Raster name:  orb25yr06ha_am
3332  = avg rainfall in inches *1000
Raster name:  orb25yr12h
3828  = avg rainfall in inches *1000
Raster name:  orb25yr12ha_am
3812  = avg rainfall in inches *1000
Raster name:  orb25yr24h
4422  = avg rainfall in inches *1000
Raster name:  orb25yr24ha_am
4404  = avg rainfall in inches *1000
Raster name:  orb2yr06h
1943  = avg rainfall in inches *1000
Raster name:  orb2yr06ha_am
1789  = a

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 010568: Invalid extent. Please check for zero length or width, or failure to project extent to output spatial reference. 
Failed to execute (ZonalStatistics).


In [11]:
st = time()
outputs = {'subwatershed': subwatershed_list[0:21],
          'area': area_list[0:21],
          'perimeter': perimeter_list[0:21],
          'circulatory_ratio': circulatory_ratio_list[0:21],
          'relief': relief_list[0:21],
          'avg_slope': avg_slope_list[0:21],
           'watershed_length': watershed_length_list[0:21],
           'elongation_ratio': elongation_ratio_list[0:21],
           'drainage_density': drainage_density_list[0:21],
           'shape_factor': shape_factor_list[0:21],
           'relief_ratio': relief_ratio_list[0:21],
           'ruggedness': ruggedness_list[0:21],
           'aae_area': aae_list[0:21],
           'buildings_aae_count': buildings_aae_list[0:21],
           'x_area': x_list[0:21],
           'buildings_x_count': buildings_x_list[0:21],
           'water_bodies_area': water_bodies_list[0:21],
           'dams_count': dams_list[0:21],
           'bridges_count': bridges_list[0:21],
           'streets_km': streets_list[0:21],
           'railroads_km': railroads_list[0:21],
           'population': population_list[0:21],
           'population_density': population_density_list[0:21],
           'avg_median_income': avg_median_income_list[0:21],
           'housing_density': housing_density_list[0:21],
           'population_change': population_change_list[0:21],
           'dependent_population_pct': dependent_population_list[0:21],
           'dist_to_stream_avg (m)': dist_to_stream_avg_list[0:21],
           'dist_to_stream_stdev (m)': dist_to_stream_stdev_list[0:21],
           'lu_21_area' : lu_21_list[0:21],
           'lu_22_area' : lu_22_list[0:21],
           'lu_23_area' : lu_23_list[0:21],
           'lu_24_area': lu_24_list[0:21],
           'lu_41_area': lu_41_list[0:21],
           'lu_82_area': lu_82_list[0:21],
           'avg_impervious_percent': impervious_percent_list[0:21],
           'orb100yr06h': orb100yr06h_list[0:21],
           'orb100yr12h': orb100yr12h_list[0:21],
           'orb100yr24h': orb100yr24h_list[0:21],
           'orb25yr06h': orb25yr06h_list[0:21],
           'orb25yr12h': orb25yr12h_list[0:21],
           'orb25yr24h':orb25yr24h_list[0:21],
           'orb2yr06h': orb2yr06h_list[0:21],
           'orb2yr12h': orb2yr12h_list[0:21],
           'orb2yr24h': orb2yr24h_list[0:21],
           'orb50yr06h': orb50yr06h_list[0:21],
           'orb50yr12h': orb50yr12h_list[0:21],
           'orb50yr24h':orb50yr24h_list[0:21],
           'orb100yr06ha_am': orb100yr06ha_am_list[0:21],
           'orb100yr12ha_am': orb100yr12ha_am_list[0:21],
           'orb100yr24ha_am': orb100yr24ha_am_list[0:21],
           'orb25yr06ha_am': orb25yr06ha_am_list[0:21],
           'orb25yr12ha_am': orb25yr12ha_am_list[0:21],
           'orb25yr24ha_am': orb25yr24ha_am_list[0:21],
           'orb2yr06ha_am': orb2yr06ha_am_list[0:21],
           'orb2yr12ha_am': orb2yr12ha_am_list[0:21],
           'orb2yr24ha_am': orb2yr24ha_am_list[0:21],
           'orb50yr06ha_am': orb50yr06ha_am_list[0:21],
           'orb50yr12ha_am': orb50yr12ha_am_list[0:21],
           'orb50yr24ha_am': orb50yr24ha_am_list[0:21]
          }

# 
# 
# 
#  
#         
# 
            
outputs_df = pd.DataFrame(outputs, columns = ['subwatershed',
                                             'area',
                                             'perimeter',
                                             'circulatory_ratio',
                                             'relief',
                                             'avg_slope',
                                              'watershed_length',
                                              'elongation_ratio',
                                              'drainage_density',
                                              'shape_factor',
                                              'relief_ratio',
                                              'ruggedness',
                                              'aae_area',
                                              'buildings_aae_count',
                                              'x_area',
                                              'buildings_x_count',
                                              'water_bodies_area',
                                              'dams_count',
                                              'bridges_count',
                                              'streets_km',
                                              'railroads_km',
                                              'population',
                                              'population_density',
                                              'avg_median_income',
                                              'housing_density',
                                              'population_change',
                                              'dependent_population_pct',
                                              'dist_to_stream_avg (m)',
                                              'dist_to_stream_stdev (m)',
                                              'lu_21_area',
                                              'lu_22_area',
                                              'lu_23_area',
                                              'lu_24_area',
                                              'lu_41_area',
                                              'lu_82_area',
                                              'avg_impervious_percent',
                                             'orb100yr06h',
                                              'orb100yr12h',
                                              'orb100yr24h',
                                              'orb25yr06h',
                                              'orb25yr12h',
                                              'orb25yr24h',
                                              'orb2yr06h',
                                              'orb2yr12h',
                                              'orb2yr24h',
                                              'orb50yr06h',
                                              'orb50yr12h',
                                              'orb50yr24h',
                                              'orb100yr06ha_am',
                                              'orb100yr12ha_am',
                                              'orb100yr24ha_am',
                                              'orb25yr06ha_am',
                                              'orb25yr12ha_am',
                                              'orb25yr24ha_am',
                                              'orb2yr06ha_am',
                                              'orb2yr12ha_am',
                                              'orb2yr24ha_am',
                                              'orb50yr06ha_am',
                                              'orb50yr12ha_am',
                                              'orb50yr24ha_am'
                                             ])    
    

#     
#  
# 
# 
# 
#




print(outputs_df)

outputs_df.to_excel(r"C:\Users\jmatney\Documents\GitHub\IndianaRisk\data\output\Marion_County_Data_updated.xlsx")
print(round(((time()-st)/60), 2) , 'minutes to process.')

    subwatershed        area  perimeter  circulatory_ratio      relief  \
0   051202081102   57.075877  42.310790           0.400645   64.000000   
1   051202080201   83.362712  59.144456           0.299470  103.000000   
2   051201110302   71.911106  43.130187           0.485784   53.000000   
3   051201060507   42.800494  40.826927           0.322675   35.000000   
4   071200010303   68.063067  51.205475           0.326203   15.000000   
5   050800030717   81.071752  51.841626           0.379072   76.000000   
6   041000030604   49.197428  49.855776           0.248726    8.964989   
7   051401010601   50.858810  30.906389           0.669082  117.000000   
8   051401040403  119.022877  63.353977           0.372643   97.000000   
9   051202050503   78.625596  77.094704           0.166236   64.000000   
10  051202030203   57.833175  40.461409           0.443920   62.000000   
11  040500011709   47.544684  37.167322           0.432503   14.000000   
12  051201130801   60.601127  37.83624

In [10]:
len(orb50yr24ha_am_list)

21